In [21]:
import onnx 
from onnx import shape_inference
import onnxruntime  as rt
import pickle

In [22]:
model =  onnx.load("ten_vad_exported.onnx")
model = shape_inference.infer_shapes(model)

In [23]:
new_outputs = []
for val_info in model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    model.graph.output.append(o)

onnx.save(model, "debug_torch_model.onnx")


In [24]:
orig_model =  onnx.load("src/onnx_model/ten-vad.onnx")
orig_model = shape_inference.infer_shapes(orig_model)

new_outputs = []
for val_info in orig_model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    orig_model.graph.output.append(o)

onnx.save(orig_model, "debug_onnx_model.onnx")

In [18]:
model_name = "debug_torch_model.onnx"
model = onnx.load(model_name)
with open("pickle_input.pkl", "rb") as f:
    input_t = pickle.load(f)

session = rt.InferenceSession(model_name, providers=['CPUExecutionProvider'])
output_names = [ o.name for o in session.get_outputs()]
raw_outputs = session.run(None, input_t)
outputs_dict = dict(zip(output_names, raw_outputs))
outputs_dict.keys()
outputs_dict['/conv_dw/Conv_output_0'].shape


(1, 1, 1, 39)